In [0]:
%run "./Config"

In [0]:
# Mounting Directory for Yelp JSONs Intake
storageAccount = "gen10datafund2205"
storageContainer = "group1-marketing-analytics"
mount_point = "/mnt/group1/tipin"
    
    
configs2 = {"fs.azure.account.auth.type": "OAuth",
   "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
   "fs.azure.account.oauth2.client.id": clientid,
   "fs.azure.account.oauth2.client.secret": clientSecret,
   "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/d46b54b2-a652-420b-aa5a-2ef7f8fc706e/oauth2/token",
   "fs.azure.createRemoteFileSystemDuringInitialization": "true"}

try:
    dbutils.fs.unmount(mount_point)
except:
    pass


dbutils.fs.mount(
source = f"abfss://{storageContainer}@{storageAccount}.dfs.core.windows.net/", 
mount_point = mount_point, 
extra_configs = configs2)

/mnt/group1/tipin has been unmounted.
Out[1]: True

In [0]:
tip_data = spark.read.format('json').options(lines=True).load('/mnt/group1/tipin/yelp_academic_dataset_tip.json')
tip_data.show()

+--------------------+----------------+-------------------+--------------------+--------------------+
 business_id|compliment_count| date| text| user_id|
+--------------------+----------------+-------------------+--------------------+--------------------+
3uLgwr0qeCNMjKenH...| 0|2012-05-18 02:17:21|Avengers time wit...|AGNUgVwnZUey3gcPC...|
QoezRbYQncpRqyrLH...| 0|2013-02-05 18:35:10|They have lots of...|NBN4MgHP9D3cw--Sn...|
MYoRNLb5chwjQe3c_...| 0|2013-08-18 00:56:08|It's open even wh...|-copOvldyKh1qr-vz...|
hV-bABTK-glh5wj31...| 0|2017-06-27 23:05:38|Very decent fried...|FjMQVZjSqY8syIO-5...|
_uN0OudeJ3Zl_tf6n...| 0|2012-10-06 19:43:09|Appetizers.. plat...|ld0AperBXk1h6Ubqm...|
7Rm9Ba50bw23KTA8R...| 0|2012-03-13 04:00:52|Chili Cup + Singl...|trf3Qcz8qvCDKXiTg...|
kH-0iXqkL7b8UXNpg...| 0|2013-12-03 23:42:15|Saturday, Dec 7th...|SMGAlRjyfuYu-c-22...|
jtri188kuhe_AuEOJ...| 0|2016-11-22 22:14:58|This is probably ...|YVBB9g23nuVJ0u44z...|
xODBZmX4EmlVvbqtK...| 0|2012-07-27 01:48:24| Tacos|VL12EhEdT4OWqGq0n...|
pICJRcyqW1cF96Q3X...| 0|2012-06-09 22:57:04|Starbucks substit...|4ay-fdVks5WMerYL_...|
clwjLY7PdYJpe7IP9...| 0|2014-06-17 01:20:14|Order the Tortill...|OttfcRxgRrYsTg9EV...|
wLHodvVFLTgK3nl2X...| 0|2017-03-23 22:01:41|Very good will de...|JsXhBw6MntzTJjH_U...|
wUMuvdUeVZODZk7Tj...| 0|2013-02-28 02:05:54|If the Hotlight i...|Y0JfJh4B-jrtGc_AH...|
MDr7KLYSPkEonvGoj...| 0|2011-07-20 21:52:57| Let's go Yankees!|MlnuJ7T14CE0JDK2Z...|
aK6R2akvIK9ijw3Fv...| 0|2014-06-12 17:34:20|Basically same fo...|ffWWVlmsrN5lZ6sjA...|
EXYbKA1tocvOK_1tX...| 0|2011-10-13 03:15:15|Don't go for dinn...|j2sEA3hiUcwHfq9Ml...|
H9fkf4Xkj_j7Zxs1F...| 0|2012-03-11 23:16:12|30 mins for take ...|jsaN4TDygu76AGTiB...|
ReX09lhufLTAx19kr...| 0|2013-06-10 20:18:41|Got the grilled c...|kjFgyrCvmVVGSlgWz...|
c5nLy7YgXG-IIrOmq...| 0|2016-04-23 02:44:03|This is the bomb ...|I6aRZ4sE7ixv0_2r3...|
LJaR65ALpz261_dlV...| 0|2012-06-02 14:39:28|Helping Mona find...|Ll5l4WTKPH7zWQWA6...|
+--------------------+----------------+-------------------+--------------------+--------------------+
only showing top 20 rows

In [0]:
tip_data = tip_data.dropna().dropDuplicates()

In [0]:
tip_data = tip_data.withColumnRenamed("business_id", "BusinessID") \
       .withColumnRenamed("compliment_count", "ComplimentCount") \
       .withColumnRenamed("date", "Date") \
       .withColumnRenamed("text", "Text") \
       .withColumnRenamed("user_id", "UserID")

In [0]:
# Write DataFrame to JSON file
tip_data.write.format('json').mode('overwrite').save(
    '/mnt/group1/tipin/tip_cleaned.json')